In [1]:
import guidance
from llama_quantized import LLaMAQuantized

/home/andy/miniconda3/envs/guider/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
guidance.llm = LLaMAQuantized(model_dir='models', model='Wizard-Vicuna-13B-Uncensored-GPTQ')

Found the following quantized model: models/Wizard-Vicuna-13B-Uncensored-GPTQ/Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors


/home/andy/miniconda3/envs/guider/lib/python3.11/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


Model to device
Loading tokenizer from: models/Wizard-Vicuna-13B-Uncensored-GPTQ/


In [3]:
preamble = '''Consider an AI named ASSISTANT, which is similar to Siri or Alexa, but much more powerful and capable.

In conversations, ASSISTANT _always_ emits output in XML, following this format:

<thought>
(some internal thought by the ASSISTANT)
</thought>
<action>
(invocation of a tool)
</action>
<output>
(output of the tool)
</output>
<response>
(final output returned as a message)
</response>

Here is an exhaustive list of valid ACTIONs that can be used during the conversation:
- WEB_SEARCH // Searches online for information to answer a question
- NONE // No action is taken; ASSISTANT simply gives a friendly reply

For _any_ factual information, ASSISTANT always defers to WEB_SEARCH, since ASSISTANT's knowledge was cutoff in 2021.

Here is an example conversation, demonstrating the rules:

=== BEGIN EXAMPLE ===
USER: Hello, are you there?
ASSISTANT: <thought>
    No ACTION is needed here.
</thought>
<action>
    NONE
</action>
<output>
</output>
<response>
    I'm here! What can I help you with?
</response></s>
USER: What are some good anime airing this season?
ASSISTANT: <thought>
    I should use WEB_SEARCH to answer this.
</thought>
<action>
    WEB_SEARCH(popular anime spring 2023)
</action>
<output>
    *omitted in example*
</output>
<response>
    There are several anime airing this season. Some of the most popular ones include:
    - Chainsaw Man
    - Bleach: Thousand-Year Blood War
    - Cyberpunk: Edgerunners
    - My Hero Academia
    - JoJo's Bizarre Adventure
    - Mob Psycho 100
</response></s>
USER: Thanks! What about next season?
ASSISTANT: <thought>
    I should use WEB_SEARCH to answer this.
</thought>
<action>
    WEB_SEARCH(popular upcoming anime)
</action>
<output>
    *omitted in example*
</output>
<response>
    Here are some of the most anticipated anime series of Winter 2024:
    - Kaiju No. 8
    - Beastars Season 3
    - Moonrise
    - Sound! Euphonium Season 3
</response></s>
USER: Great, thank you.
ASSISTANT: <thought>
    No ACTION is needed here.
</thought>
<action>
    NONE
</action>
<output>
</output>
<response>
    Of course! Anything else I can help you with?
</response></s>
=== END EXAMPLE ===

Below is a never-ending interaction between ASSISTANT and USER, following the convention described above.

CURRENT DATE: May 14, 2023

// Enabled ACTIONs for this conversation: [ WEB_SEARCH, NONE ]
Begin!'''

history = guidance('')

chat = guidance('''
{{~preamble}}
{{history}}
{{~#user~}}
{{user_input}}
{{~/user}}
{{#assistant~}}
<thought>{{gen 'thought' stop='</thought>'}}</thought>
<action>
    {{select 'action' options=valid_actions}}({{gen 'action_input'}})
</action>
<output>
    {{await 'output'}}
</output>
<response>{{gen 'response' stop='</response>'}}</response>
{{~/assistant}}
''')


preamble_len = len(preamble)
history_text = history.text
if len(history_text) >= preamble_len:
    history_text = history_text[preamble_len:].lstrip()

valid_actions = ['WEB_SEARCH,' 'NONE']

user_input = "what are some good movies to watch on a cozy, rainy night?"

output_step = chat(preamble=preamble, history=history_text, user_input=user_input, valid_actions=valid_actions)


Stop program Consider an AI named ASSISTANT, which is similar to Siri or Alexa, but much more powerful and capable.

In conversations, ASSISTANT _always_ emits output in XML, following this format:

<thought>
(some internal thought by the ASSISTANT)
</thought>
<action>
(invocation of a tool)
</action>
<output>
(output of the tool)
</output>
<response>
(final output returned as a message)
</response>

Here is an exhaustive list of valid ACTIONs that can be used during the conversation:
- WEB_SEARCH // Searches online for information to answer a question
- NONE // No action is taken; ASSISTANT simply gives a friendly reply

For _any_ factual information, ASSISTANT always defers to WEB_SEARCH, since ASSISTANT's knowledge was cutoff in 2021.

Here is an example conversation, demonstrating the rules:

=== BEGIN EXAMPLE ===
USER: Hello, are you there?
ASSISTANT: <thought>
 No ACTION is needed here.
</thought>
<action>
 NONE
</action>
<output>
</output>
<response>
 I'm here! What can I help you with?
</response></s>
USER: What are some good anime airing this season?
ASSISTANT: <thought>
 I should use WEB_SEARCH to answer this.
</thought>
<action>
 WEB_SEARCH(popular anime spring 2023)
</action>
<output>
 *omitted in example*
</output>
<response>
 There are several anime airing this season. Some of the most popular ones include:
 - Chainsaw Man
 - Bleach: Thousand-Year Blood War
 - Cyberpunk: Edgerunners
 - My Hero Academia
 - JoJo's Bizarre Adventure
 - Mob Psycho 100
</response></s>
USER: Thanks! What about next season?
ASSISTANT: <thought>
 I should use WEB_SEARCH to answer this.
</thought>
<action>
 WEB_SEARCH(popular upcoming anime)
</action>
<output>
 *omitted in example*
</output>
<response>
 Here are some of the most anticipated anime series of Winter 2024:
 - Kaiju No. 8
 - Beastars Season 3
 - Moonrise
 - Sound! Euphonium Season 3
</response></s>
USER: Great, thank you.
ASSISTANT: <thought>
 No ACTION is needed here.
</thought>
<action>
 NONE
</action>
<output>
</output>
<response>
 Of course! Anything else I can help you with?
</response></s>
=== END EXAMPLE ===

Below is a never-ending interaction between ASSISTANT and USER, following the convention described above.

CURRENT DATE: May 14, 2023

// Enabled ACTIONs for this conversation: [ WEB_SEARCH, NONE ]
Begin! 
 user what are some good movies to watch on a cozy, rainy night? assistant <thought> 
 I should use WEB_SEARCH to answer this.
 </thought>
<action>
 WEB_SEARCH,NONE ( cozy,rainy night movies )
</action>
<output>
 {{await 'output'}} 
</output>
<response> {{gen 'response' stop='</response>'}} </response>

In [4]:
output_step.variables()

{'llm': <llama_quantized.LLaMAQuantized at 0x7fa63a57d490>,
 'preamble': "Consider an AI named ASSISTANT, which is similar to Siri or Alexa, but much more powerful and capable.\n\nIn conversations, ASSISTANT _always_ emits output in XML, following this format:\n\n<thought>\n(some internal thought by the ASSISTANT)\n</thought>\n<action>\n(invocation of a tool)\n</action>\n<output>\n(output of the tool)\n</output>\n<response>\n(final output returned as a message)\n</response>\n\nHere is an exhaustive list of valid ACTIONs that can be used during the conversation:\n- WEB_SEARCH // Searches online for information to answer a question\n- NONE // No action is taken; ASSISTANT simply gives a friendly reply\n\nFor _any_ factual information, ASSISTANT always defers to WEB_SEARCH, since ASSISTANT's knowledge was cutoff in 2021.\n\nHere is an example conversation, demonstrating the rules:\n\n=== BEGIN EXAMPLE ===\nUSER: Hello, are you there?\nASSISTANT: <thought>\n    No ACTION is needed here.\n<